# Number of suicides per city
We extracted data from DATASUS TabNet interface, corresponding to the following parameters:

- Óbitos p/Residênc por Ano do Óbito segundo Município
- Grupo CID-10: Lesões autoprovocadas intencionalmente
- Período: 2008-2018

The data can be found at CSV/Tabnet/suicides_08_2018.csv


In [1]:
import pandas as pd
root = "../"

Read file with suicide data

In [16]:
suicide_data = pd.read_csv(root + 'CSV/TabNet/suicides_08_18.csv', sep=";")
suicide_data

,Município,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Total
0,110001 Alta Floresta D'Oeste,5,2,2,1,3,2,1,3,2,3,1,25
1,110037 Alto Alegre dos Parecis,-,3,2,-,3,1,1,-,2,1,2,15
2,110040 Alto Paraíso,1,1,1,1,1,-,-,-,-,1,2,8
3,110034 Alvorada D'Oeste,-,-,1,1,-,2,-,1,1,-,2,8
4,110002 Ariquemes,8,2,4,4,9,2,5,5,6,10,2,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5399,522205 Vicentinópolis,-,-,-,-,1,1,-,1,-,-,-,3
5400,522220 Vila Boa,3,-,-,-,-,-,-,-,-,-,-,3
5401,522230 Vila Propício,-,1,-,-,-,1,-,-,-,-,-,2
5402,520000 Município ignorado - GO,1,3,2,2,1,2,2,2,1,3,-,19


Replace "-" values with 0, create a column with the city code (`MUNCOD`)

In [17]:
suicide_data['MUNCOD'] = suicide_data['Município'].str.split(' ', expand=True)[0]
suicide_data.drop('Município', axis=1, inplace=True)
suicide_data = suicide_data.replace("-", 0)
suicide_data

,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,Total,MUNCOD
0,5,2,2,1,3,2,1,3,2,3,1,25,110001
1,0,3,2,0,3,1,1,0,2,1,2,15,110037
2,1,1,1,1,1,0,0,0,0,1,2,8,110040
3,0,0,1,1,0,2,0,1,1,0,2,8,110034
4,8,2,4,4,9,2,5,5,6,10,2,57,110002
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5399,0,0,0,0,1,1,0,1,0,0,0,3,522205
5400,3,0,0,0,0,0,0,0,0,0,0,3,522220
5401,0,1,0,0,0,1,0,0,0,0,0,2,522230
5402,1,3,2,2,1,2,2,2,1,3,0,19,520000


Save data to a .csv

In [18]:
suicide_data.to_csv(root + 'CSV/Suicide/suicide_count_08_18.csv')

# Rate of suicides per city

## Part I: get population data
Source: ftp://ftp.datasus.gov.br/dissemin/publicos/IBGE/POPTCU/

In [63]:
import urllib.request
from zipfile import ZipFile
import os
import dbf

def get_pop_data(year):
    pop_zip = 'POPTBR' + year + '.zip'
    pop_file = 'POPTBR' + year + '.csv'
    url = 'ftp://ftp.datasus.gov.br/dissemin/publicos/IBGE/POPTCU/' + pop_zip
    urllib.request.urlretrieve(url, 'temp/' + pop_zip)
    with ZipFile('temp/' + pop_zip, 'r') as zipObj:
        zipObj.extractall(path="temp/")
        
        if int(year) > 13:
            fname = 'temp/' + zipObj.namelist()[0]
            with dbf.Table(fname) as table:
                dbf.export(table, 'temp/' + pop_file)
    
    df = pd.read_csv('temp/' + pop_file)
    df = df.set_axis(['MUNCOD', 'ANO', 'POPULACAO'], axis=1, inplace=False)
    df = df[["POPULACAO", "MUNCOD"]]
    df = df[df["POPULACAO"].apply(lambda x: isinstance(x, int) or x.isnumeric())]
    df["POPULACAO"] = df["POPULACAO"].astype('int64')
    df["MUNCOD"] = df["MUNCOD"].astype('int64')
    df["MUNCOD"] = [int(str(x)[:6]) for x in df['MUNCOD']]
    
    df.rename(columns={"POPULACAO": "POP_" + year}, inplace=True)
    return df

In [64]:
all_years = pd.DataFrame()
years = ["08", "09", "10", "11", "12", "13", "14", "15", "16", "17", "18"]
for year in years:
    df = get_pop_data(year)
    if "MUNCOD" in all_years:
        all_years = pd.merge(df, all_years, left_on="MUNCOD",right_on="MUNCOD")
    else:
        all_years = df
all_years

,POP_18,MUNCOD,POP_17,POP_16,POP_15,POP_14,POP_13,POP_12,POP_11,POP_10,POP_09,POP_08
0,23167,110001,25437,25506,25578,25652,25728,24069,24228,24422,24354,24577
1,106168,110002,107345,105896,104401,102860,101269,92747,91570,90354,85541,84581
2,5438,110003,6224,6289,6355,6424,6495,6132,6221,6309,6695,6777
3,84813,110004,88507,87877,87226,86556,85863,79330,78959,78601,78675,78263
4,16444,110005,17934,17959,17986,18013,18041,16852,16939,17030,16622,16784
...,...,...,...,...,...,...,...,...,...,...,...,...
5560,13746,522200,13675,13567,13456,13343,13227,12737,12644,12549,12831,12699
5561,8611,522205,8397,8286,8171,8053,7933,7576,7476,7371,6093,6091
5562,6026,522220,5731,5615,5495,5371,5246,4954,4847,4742,4578,4461
5563,5758,522230,5690,5635,5578,5520,5460,5244,5196,5145,5359,5282


Saving it to a .csv file

In [65]:
all_years.to_csv(root + 'CSV/Population/population_08_18.csv')

Deleting all temporary files

In [66]:
import os
import glob

files = glob.glob('temp/*')
for f in files:
    os.remove(f)

## Part II: calculate rates

In [72]:
population = pd.read_csv(root + 'CSV/Population/population_08_18.csv', index_col=[0])
suicides = pd.read_csv(root + 'CSV/Suicide/suicide_count_08_18.csv', index_col=[0])
df = pd.merge(population, suicides, left_on="MUNCOD", right_on="MUNCOD")
df = df.drop(['Total'], axis=1)
df

,POP_18,MUNCOD,POP_17,POP_16,POP_15,POP_14,POP_13,POP_12,POP_11,POP_10,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,23167,110001,25437,25506,25578,25652,25728,24069,24228,24422,...,2,2,1,3,2,1,3,2,3,1
1,106168,110002,107345,105896,104401,102860,101269,92747,91570,90354,...,2,4,4,9,2,5,5,6,10,2
2,5438,110003,6224,6289,6355,6424,6495,6132,6221,6309,...,1,0,0,0,1,0,2,1,0,1
3,84813,110004,88507,87877,87226,86556,85863,79330,78959,78601,...,6,2,1,4,1,5,6,9,5,8
4,16444,110005,17934,17959,17986,18013,18041,16852,16939,17030,...,0,2,0,2,0,0,2,2,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5376,13746,522200,13675,13567,13456,13343,13227,12737,12644,12549,...,0,2,2,2,0,1,0,0,1,0
5377,8611,522205,8397,8286,8171,8053,7933,7576,7476,7371,...,0,0,0,1,1,0,1,0,0,0
5378,6026,522220,5731,5615,5495,5371,5246,4954,4847,4742,...,0,0,0,0,0,0,0,0,0,0
5379,5758,522230,5690,5635,5578,5520,5460,5244,5196,5145,...,1,0,0,0,1,0,0,0,0,0


Calculate rates

In [73]:
for year in years:
    df['RATE_' + year] = df['20' + year]/df['POP_' + year] * 100000
    df = df.drop(['20' + year, 'POP_' + year], axis=1)
df

,MUNCOD,RATE_08,RATE_09,RATE_10,RATE_11,RATE_12,RATE_13,RATE_14,RATE_15,RATE_16,RATE_17,RATE_18
0,110001,20.344224,8.212203,8.189337,4.127456,12.464166,7.773632,3.898332,11.728829,7.841292,11.793844,4.316485
1,110002,9.458389,2.338060,4.427031,4.368243,9.703818,1.974938,4.860976,4.789226,5.665936,9.315758,1.883807
2,110003,0.000000,14.936520,0.000000,0.000000,0.000000,15.396459,0.000000,31.471282,15.900779,0.000000,18.389114
3,110004,5.110972,7.626311,2.544497,1.266480,5.042229,1.164646,5.776607,6.878683,10.241588,5.649271,9.432516
4,110005,0.000000,0.000000,11.743981,0.000000,11.868028,0.000000,0.000000,11.119760,11.136478,5.576001,6.081245
...,...,...,...,...,...,...,...,...,...,...,...,...
5376,522200,7.874636,0.000000,15.937525,15.817779,15.702285,0.000000,7.494566,0.000000,0.000000,7.312614,0.000000
5377,522205,0.000000,0.000000,0.000000,0.000000,13.199578,12.605572,0.000000,12.238404,0.000000,0.000000,0.000000
5378,522220,67.249496,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5379,522230,0.000000,18.660198,0.000000,0.000000,0.000000,18.315018,0.000000,0.000000,0.000000,0.000000,0.000000


Export to csv

In [74]:
df.to_csv(root + 'CSV/Suicide/suicide_rates_08_18.csv')